<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/NER_ADE_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def readfile():
  path='/content/drive/MyDrive/NLP/ADE/Dataset/ADEtrain.tsv'
  f=open(path)
  data1=[]
  data = []
  sentence=[]
  label=[]
  for line in f:
  #print(line)
  #print(len(line))
    line1=line.split('\t')
  #print(len(line1))
  #print(line1)
    if line1[0]=="\n" or len(line1)==0:
      if len(sentence) >0:
      #print("Sentence Length",)
    #if len(sentence) != 0:
      #print("New line")
    #print(line1)
        data1.append((sentence,label))
        data=[]
        sentence=[]
        label=[]
      continue
    #else:
      #continue
    elif len(line1)<=5:    
      #print("other")
      #print(line1)
      continue 
    else:
    #line1=line.split('\t')
      #print(line1)
      sentence.append(line1[5])
      label.append(line1[1])
  return data1    

In [3]:
#Call the functiona
data1=readfile()

In [4]:
#Print a sample data
data1[0]

(['Intravenous', 'azithromycin', '-', 'induced', 'ototoxicity', '.'],
 ['O', 'S-DRUG', 'O', 'O', 'S-AE', 'O'])

In [5]:
#Length of the sentence
leng=[]
for i in range(len(data1)):
  leng.append(len(data1[i][0]))
#print(len(leng))
print(sum(leng)/len(leng))


21.599762752075918


In [6]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [7]:
import pandas as pd
import csv

In [8]:
tags=['O','B-AE','I-AE','E-AE','S-AE','B-DRUG','I-DRUG','E-DRUG','S-DRUG','PAD']
labels_map={}
for i,t in enumerate(tags):
  labels_map[t]=i
print(labels_map)

{'O': 0, 'B-AE': 1, 'I-AE': 2, 'E-AE': 3, 'S-AE': 4, 'B-DRUG': 5, 'I-DRUG': 6, 'E-DRUG': 7, 'S-DRUG': 8, 'PAD': 9}


In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [10]:
!pip3 install transformers

     |████████████████████████████████| 1.4MB 8.1MB/s 
     |████████████████████████████████| 2.9MB 44.6MB/s 
     |████████████████████████████████| 890kB 39.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d9e0da94bd066d40f7a4216348f186c7cfd82a7b6822dacd43c397f50752cea0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
#Apply the Bert
#use pytorch implementation of bert
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModel
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [12]:
#download the biobert models from dmis lab - korea univ
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")

12/09/2020 10:40:56 - INFO - filelock -   Lock 140083959919000 acquired on /root/.cache/huggingface/transformers/f048b8136bae2b3abe91e9e82949295fb205887c84db3be2775e1cdb0ecfeeb9.d7812d36d3371e4d43299a0c4a938622c5251db0efa17a5d4d9b57037fcec823.lock


12/09/2020 10:40:57 - INFO - filelock -   Lock 140083959919000 released on /root/.cache/huggingface/transformers/f048b8136bae2b3abe91e9e82949295fb205887c84db3be2775e1cdb0ecfeeb9.d7812d36d3371e4d43299a0c4a938622c5251db0efa17a5d4d9b57037fcec823.lock


12/09/2020 10:40:57 - INFO - filelock -   Lock 140083959918720 acquired on /root/.cache/huggingface/transformers/cda52d3a8283b321708097045e27f11cd70bbf3ad8cdefa2c0a56f187855f5d5.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock


12/09/2020 10:40:57 - INFO - filelock -   Lock 140083959918720 released on /root/.cache/huggingface/transformers/cda52d3a8283b321708097045e27f11cd70bbf3ad8cdefa2c0a56f187855f5d5.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock


12/09/2020 10:40:58 - INFO - filelock -   Lock 140081533726224 acquired on /root/.cache/huggingface/transformers/118da8438a7854000cfcf052566f83ae4f4159ac25796e49e16c3b18746041b4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


12/09/2020 10:40:58 - INFO - filelock -   Lock 140081533726224 released on /root/.cache/huggingface/transformers/118da8438a7854000cfcf052566f83ae4f4159ac25796e49e16c3b18746041b4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


12/09/2020 10:40:58 - INFO - filelock -   Lock 140081533726224 acquired on /root/.cache/huggingface/transformers/2fb6c5805404829e9c10c33b38ae59ae3011225799f3177f769a06a7411fa46c.25d8d06fb0679146a3ed2a3463e3585380bff882fe6e1ebc497196e40dbbd7fa.lock


12/09/2020 10:40:59 - INFO - filelock -   Lock 140081533726224 released on /root/.cache/huggingface/transformers/2fb6c5805404829e9c10c33b38ae59ae3011225799f3177f769a06a7411fa46c.25d8d06fb0679146a3ed2a3463e3585380bff882fe6e1ebc497196e40dbbd7fa.lock


12/09/2020 10:40:59 - INFO - filelock -   Lock 140081533513512 acquired on /root/.cache/huggingface/transformers/65231a5792b14eb81b9a6bdccccfffda18575eb3bafbb730c9fa4235e56c3c17.74cc2087932cb523a583bd5e65732ee1aaade59dfc0b62f88101de7567d92e42.lock


12/09/2020 10:41:11 - INFO - filelock -   Lock 140081533513512 released on /root/.cache/huggingface/transformers/65231a5792b14eb81b9a6bdccccfffda18575eb3bafbb730c9fa4235e56c3c17.74cc2087932cb523a583bd5e65732ee1aaade59dfc0b62f88101de7567d92e42.lock


In [13]:
#convert the sequences to tokens 
sentences=[]
label=[]
for i in data1:
  sentence=i[0]
  tags=i[1]
  tokenized_sentence=[]
  labels=[]
  for word,tag in zip(sentence,tags):      
    tokenized_word=tokenizer.tokenize(word)
    #print(tokenized_word)
    n_subwords=len(tokenized_word)
    tokenized_sentence.extend(tokenized_word)
    labels.extend([tag] * n_subwords)
  sentences.append(tokenized_sentence)
  label.append(labels)

In [14]:
print(len(sentences))

4215


In [15]:
print(len(label))

4215


In [16]:
#print a sample sentence
sentences[5]

['R',
 '##ES',
 '##U',
 '##LT',
 '##S',
 ':',
 'A',
 '44',
 '-',
 'year',
 '-',
 'old',
 'man',
 'taking',
 'nap',
 '##ro',
 '##xen',
 'for',
 'chronic',
 'low',
 'back',
 'pain',
 'and',
 'a',
 '20',
 '-',
 'year',
 '-',
 'old',
 'woman',
 'on',
 'o',
 '##xa',
 '##p',
 '##ro',
 '##zin',
 'for',
 'r',
 '##he',
 '##uma',
 '##to',
 '##id',
 'art',
 '##hr',
 '##itis',
 'presented',
 'with',
 'tense',
 'bull',
 '##ae',
 'and',
 'cut',
 '##aneous',
 'f',
 '##rag',
 '##ility',
 'on',
 'the',
 'face',
 'and',
 'the',
 'back',
 'of',
 'the',
 'hands',
 '.']

In [17]:
#print a sample label
label[5]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-DRUG',
 'S-DRUG',
 'S-DRUG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-DRUG',
 'S-DRUG',
 'S-DRUG',
 'S-DRUG',
 'S-DRUG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-AE',
 'E-AE',
 'E-AE',
 'O',
 'B-AE',
 'B-AE',
 'E-AE',
 'E-AE',
 'E-AE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [18]:
#pad_sequences is used to ensure that all sequences in a list have the same length
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [19]:
#setting the maximum size to be 128 since only few sequences have above 128
MAX_LEN = 128
#keep the batch size as 16 
Batch_Size = 16

In [20]:
#Convert tokens to ids and pad with 0's till the max seq length of 128
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in sentences],maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [21]:
label[2][5]

'S-AE'

In [ ]:
label[2][5]='B-AE'

In [ ]:
p=[[labels_map.get(l) for l in lab] for lab in label]
p[2]

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [22]:
#Convert tags to ids and pad with 0's till the max seq length of 128
tags = pad_sequences([[labels_map.get(l) for l in lab] for lab in label],
                     maxlen=MAX_LEN, value=labels_map["PAD"], padding="post",
                     dtype="long", truncating="post")

In [25]:
tags[56]

array([0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 1, 3, 3, 3, 3, 0, 4, 4,
       4, 4, 0, 0, 0, 4, 4, 4, 0, 0, 0, 8, 0, 0, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [26]:
input_ids[56]

array([22157, 24162, 15609,  1708,   131,  1109,  5351,  2320, 26825,
        3532,  5026, 17030,  8569,  1111,  5199,   190,  3740,  4578,
        3464,   117,  1126, 10712, 15018,  1918,   117,  1105,  7957,
         192, 19989,  6185,  3694,  1121,  1123, 26825, 13753,   119,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [27]:
#attenton masks to ensure padding trainin is not impacted
attention_masks = [[float(i!= 0.0) for i in ii] for ii in input_ids]

In [28]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018, test_size=0.3)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.3)

In [29]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [30]:
val_inputs=val_inputs[:1264]
val_tags=val_tags[:1264]
val_masks=val_masks[:1264]

In [31]:
val_inputs.shape

torch.Size([1264, 128])

In [32]:
#train_data in the form - input tokens, attention masks for the corresponding tokens and the tags for the tokens
train_data = TensorDataset(tr_inputs,tr_masks,tr_tags)
#returns random indices from the data source
train_sampler = RandomSampler(train_data)
#dataloader combines the dataset and the sampler 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=Batch_Size)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
#samples elements sequentially in the same order
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,sampler=valid_sampler, batch_size=Batch_Size)

In [33]:
val_inputs.shape

torch.Size([1264, 128])

In [34]:
import transformers
#bertfortokenclassification addsa classifier on top of bert model
from transformers import AutoModelForTokenClassification,AdamW,AutoConfig

In [35]:
print(len(labels_map))

10


In [36]:
from transformers import BertForTokenClassification,AdamW,AutoConfig

model2 = BertForTokenClassification.from_pretrained("dmis-lab/biobert-v1.1",num_labels=len(labels_map), output_attentions = False,output_hidden_states = False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [38]:
# Tell pytorch to run this model on the GPU.
model2.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [39]:
model2.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [40]:
#grab the optimzer
optimizer=AdamW(model2.parameters(),
               lr=2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 
               eps=1e-8) #a very small number to prevent any division by zero in the implementation)

In [41]:
from transformers import get_linear_schedule_with_warmup

In [42]:
epochs = 3

In [43]:
total_steps = len(train_dataloader) * epochs

In [44]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,#deafault value
    num_training_steps=total_steps)


In [45]:
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix

In [46]:
import random
import numpy as np

In [47]:
# Set the seed value all over the place to make this reproducible.
seed_val=42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [48]:
def get_special_tokens(tokenizer, labels_map):
  pad_tok = tokenizer.vocab["[PAD]"]
  sep_tok = tokenizer.vocab["[SEP]"]
  cls_tok = tokenizer.vocab["[CLS]"]
  o_lab = labels_map["O"]
  return pad_tok, sep_tok, cls_tok, o_lab

In [49]:
def flat_accuracy(valid_tags, pred_tags):

    """
    Define a flat accuracy metric to use while training the model.
    """

    return (np.array(valid_tags) == np.array(pred_tags)).mean()

In [50]:
def annot_confusion_matrix(valid_tags, pred_tags):

    """
    Create an annotated confusion matrix by adding label
    annotations and formatting to sklearn's `confusion_matrix`.
    """

    # Create header from unique tags
    header = sorted(list(set(valid_tags + pred_tags)))

    # Calculate the actual confusion matrix
    matrix = confusion_matrix(valid_tags, pred_tags, labels=header)

    # Final formatting touches for the string output
    mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content


In [53]:
taglist=['O','B-AE','I-AE','E-AE','S-AE','B-DRUG','I-DRUG','E-DRUG','S-DRUG','PAD']

In [54]:
#check --
import time
loss_values, validation_loss_values = [], []
training_stats = []
tic= time.time()

pad_tok, sep_tok, cls_tok, o_lab = get_special_tokens(tokenizer, labels_map)

for _ in trange(epochs,desc="EPOCH"):
  print("")
  #print ---Epoch 1/2 ----
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

 
  #t0 = time.time()
  #train the model
  model2.train()
  tr_loss, tr_accuracy = 0,0
  nb_tr_examples, nb_tr_steps = 0,0
  tr_preds, tr_labels = [], []

  for nb_tr_steps,batch in enumerate(train_dataloader):
    print(nb_tr_steps)
    #print(batch)

    # Unpack this training batch from our dataloader and load to GPU
    b_input_ids = batch[0].to(device) # input ids
    b_input_mask = batch[1].to(device) # attention masks to performing attention on padding token indices
    b_labels = batch[2].to(device) # labels

    #always clear the gradients before doing a backward pass
    model2.zero_grad()
   
    #print("inputids",b_input_ids)
    #print("labels",b_labels)
    #print("Mask",b_input_mask)



    #perform a forward pass
    output = model2(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    #print(output)

    #print("loss",logits1)
    #print("loss",output[0])
    #print("logit",output[1])
    #print("logits",loss)
    loss=output[0]
    logits=output[1]

    #Perform backward loss
    loss.backward()

    #print(loss.item())
    #Add the loss
    tr_loss += loss.item()
    #Compute the totale number of examples based on the batch size
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

     # Subset out unwanted predictions on CLS/PAD/SEP tokens
    preds_mask = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok))

    pred_mask=preds_mask.to("cpu").numpy()

     #logits = logits.detach().cpu().numpy()
    tr_logits = logits.detach().cpu().numpy()

    tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))
    tr_batch_preds = np.argmax(tr_logits[pred_mask.squeeze()], axis=1)
    tr_batch_labels = tr_label_ids.to("cpu").numpy()

    tr_preds.extend(tr_batch_preds)
    tr_labels.extend(tr_batch_labels)

    # Compute training accuracy
    tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds)


    tr_accuracy += tmp_tr_accuracy

    torch.nn.utils.clip_grad_norm_(model2.parameters(), 1.0)


    #Update the weights
    optimizer.step()

    #Update the learning rate
    scheduler.step()
  
  #Compute the avergae loss
  tr_loss = tr_loss / nb_tr_steps
  #compute the average accuracy
  tr_accuracy = tr_accuracy / nb_tr_steps

  #Print the computed metrics for training after the epochs
  print(f"Train loss: {tr_loss}")
  print(f"Train accuracy: {tr_accuracy}")
  print(" Training is completed")
  print("Running Validation... ")


  ##############################
    ##### VALIDATION STARTS #####
    ############################  

  model2.eval() # getting the trained model to evaluation mode
  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  predictions, true_labels = [], []

 
  for batch in valid_dataloader:
    b_input_ids = batch[0].to(device) # input ids
    b_input_mask = batch[1].to(device) # attention masks
    b_labels = batch[2].to(device) # labels

    with torch.no_grad():
      output=model2(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

      tmp_eval_loss,logits = output[:2]

    # Subset out unwanted predictions on CLS/PAD/SEP tokens
    preds_mask = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok))

    #Derive the ligits to cpu
    logits = logits.detach().cpu().numpy()


    label_ids = torch.masked_select(b_labels,(preds_mask == 1))

    #Move the mask to cpu 
    preds_mask=preds_mask.to("cpu").numpy()



    #np.argmax 
    val_batch_preds =np.argmax(logits[preds_mask.squeeze()],axis=1)


    val_batch_labels = label_ids.to("cpu").numpy()

    predictions.extend(val_batch_preds)
    true_labels.extend(val_batch_labels)

    #compute the accuracy
    tmp_eval_accuracy = flat_accuracy(val_batch_labels, val_batch_preds)
    
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
     
    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1  

  pred_tags = [taglist[i] for i in predictions]
  valid_tags = [taglist[i] for i in true_labels]
  cl_report = classification_report(valid_tags, pred_tags)
  conf_mat = annot_confusion_matrix(valid_tags, pred_tags)
  eval_loss = eval_loss / nb_eval_steps
  eval_accuracy = eval_accuracy / nb_eval_steps

  # Report metrics
  print(f"Validation loss: {eval_loss}")
  print(f"Validation Accuracy: {eval_accuracy}")
  print(f"Classification Report:\n {cl_report}")
  print(f"Confusion Matrix:\n {conf_mat}")




   


EPOCH:   0%|          | 0/3 [00:00<?, ?it/s]


Training...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
Train loss: 0.198115436189078
Train accuracy: 0.9388901415392488
 Training is completed
Running Validation... 



EPOCH:  33%|███▎      | 1/3 [01:32<03:05, 92.95s/it]

Validation loss: 0.2259089805280106
Validation Accuracy: 0.9296295757672064
Classification Report:
               precision    recall  f1-score   support

        B-AE       0.85      0.82      0.83      2494
      B-DRUG       0.84      0.90      0.87       602
        E-AE       0.83      0.88      0.86      2500
      E-DRUG       0.80      0.89      0.84       573
        I-AE       0.75      0.77      0.76      1860
      I-DRUG       0.94      0.45      0.61       262
           O       0.97      0.96      0.96     27138
        S-AE       0.90      0.87      0.89      2322
      S-DRUG       0.93      0.98      0.95      5201

    accuracy                           0.93     42952
   macro avg       0.87      0.84      0.84     42952
weighted avg       0.93      0.93      0.93     42952

Confusion Matrix:
 	B-AE B-DRUG E-AE E-DRUG I-AE I-DRUG O S-AE S-DRUG
B-AE	[2033    4    1    0  185    0  227   24   20]
B-DRUG	[  5 539   0   0   0   1  19   0  38]
E-AE	[   0    0 2210    1   


EPOCH:  67%|██████▋   | 2/3 [03:05<01:32, 92.82s/it]

Validation loss: 0.22994692727357527
Validation Accuracy: 0.9348400447671378
Classification Report:
               precision    recall  f1-score   support

        B-AE       0.82      0.85      0.84      2494
      B-DRUG       0.86      0.94      0.90       602
        E-AE       0.84      0.90      0.87      2500
      E-DRUG       0.89      0.92      0.90       573
        I-AE       0.76      0.79      0.77      1860
      I-DRUG       0.97      0.73      0.83       262
           O       0.97      0.95      0.96     27138
        S-AE       0.91      0.88      0.89      2322
      S-DRUG       0.94      0.98      0.96      5201

    accuracy                           0.93     42952
   macro avg       0.88      0.88      0.88     42952
weighted avg       0.94      0.93      0.93     42952

Confusion Matrix:
 	B-AE B-DRUG E-AE E-DRUG I-AE I-DRUG O S-AE S-DRUG
B-AE	[2128    4    0    0  169    0  170   12   11]
B-DRUG	[  5 567   0   0   0   1  15   0  14]
E-AE	[   0    0 2245    1  


EPOCH: 100%|██████████| 3/3 [04:38<00:00, 92.67s/it]

Validation loss: 0.22994692727357527
Validation Accuracy: 0.9348400447671378
Classification Report:
               precision    recall  f1-score   support

        B-AE       0.82      0.85      0.84      2494
      B-DRUG       0.86      0.94      0.90       602
        E-AE       0.84      0.90      0.87      2500
      E-DRUG       0.89      0.92      0.90       573
        I-AE       0.76      0.79      0.77      1860
      I-DRUG       0.97      0.73      0.83       262
           O       0.97      0.95      0.96     27138
        S-AE       0.91      0.88      0.89      2322
      S-DRUG       0.94      0.98      0.96      5201

    accuracy                           0.93     42952
   macro avg       0.88      0.88      0.88     42952
weighted avg       0.94      0.93      0.93     42952

Confusion Matrix:
 	B-AE B-DRUG E-AE E-DRUG I-AE I-DRUG O S-AE S-DRUG
B-AE	[2128    4    0    0  169    0  170   12   11]
B-DRUG	[  5 567   0   0   0   1  15   0  14]
E-AE	[   0    0 2245    1  

In [58]:
test_sentence="A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."






In [56]:
#Check for tokenizer - biobert
def evaluate1(test_sentence):
  tokenized_sentence=tokenizer.encode(test_sentence)
  #conver to tensor format
  input_ids = torch.tensor([tokenized_sentence])
  input_ids=input_ids.cuda()
  print(input_ids)
  #run the sentence through the model
  with torch.no_grad():
    output=model2(input_ids)
  label_indices=torch.argmax(output[0],axis=2)
  tokens=tokenizer.convert_ids_to_tokens(input_ids[0])
  tags=['O','B-AE','I-AE','E-AE','S-AE','B-DRUG','I-DRUG','E-DRUG','S-DRUG','PAD']
  new_token,new_labels=[],[]
  for token,label_idx in zip(tokens,label_indices[0]):
  #print(token)
  #print(tags[label_idx])
    if token.startswith("##"):
    #add at the end
      new_token[-1]=new_token[-1]+token[2:]
    else:
    #if its not special token
      new_labels.append(tags[label_idx])
      new_token.append(token)
  for token,label in zip(new_token,new_labels):
    print("{}\t{}".format(token,label))   

In [59]:
evaluate1(test_sentence)

tensor([[  101,   138,  3140,   118,  1214,   118,  1385,  1299,  1781, 22956,
          2180, 25926,  1111, 13306,  1822,  1171,  2489,  1105,   170,  1406,
           118,  1214,   118,  1385,  1590,  1113,   184, 20192,  1643,  2180,
         17817,  1111,   187,  4638, 10161,  2430,  2386,  1893,  8167, 10721,
          2756,  1114,  8901, 12200,  5024,  1105,  2195, 13064,   175, 20484,
         13378,  1113,  1103,  1339,  1105,  1103,  1171,  1104,  1103,  1493,
           119,   102]], device='cuda:0')
[CLS]	O
A	O
44	O
-	O
year	O
-	O
old	O
man	O
taking	O
naproxen	S-DRUG
for	O
chronic	O
low	O
back	O
pain	O
and	O
a	O
20	O
-	O
year	O
-	O
old	O
woman	O
on	O
oxaprozin	S-DRUG
for	O
rheumatoid	O
arthritis	O
presented	O
with	O
tense	O
bullae	E-AE
and	O
cutaneous	B-AE
fragility	E-AE
on	O
the	O
face	O
and	O
the	O
back	O
of	O
the	O
hands	O
.	O
[SEP]	O


In [ ]:
model2.save_pretrained('/content/drive/MyDrive/NLP/ADE/Model-Dec7')

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/NLP/ADE/Model-Dec7')

('/content/drive/MyDrive/NLP/ADE/Model-Dec7/tokenizer_config.json',
 '/content/drive/MyDrive/NLP/ADE/Model-Dec7/special_tokens_map.json',
 '/content/drive/MyDrive/NLP/ADE/Model-Dec7/vocab.txt',
 '/content/drive/MyDrive/NLP/ADE/Model-Dec7/added_tokens.json')

In [ ]:
tags=['O','B-AE','I-AE','PAD']

In [ ]:
label_map = {i : label for i, label in enumerate(tags,1)}
print(label_map)

{1: 'O', 2: 'B-AE', 3: 'I-AE', 4: 'PAD'}


In [ ]:
print(len(label_map))

4


In [ ]:
bert_model='dmis-lab/biobert-v1.1'

In [ ]:
import json

In [ ]:
label_map = {i : label for i, label in enumerate(tags,1)}
model_config={"bert_model":bert_model,"max_seq_length":128,"num_labels":len(label_map),"label_map":label_map}

In [ ]:
model_config

{'bert_model': 'dmis-lab/biobert-v1.1',
 'label_map': {1: 'O', 2: 'B-AE', 3: 'I-AE', 4: 'PAD'},
 'max_seq_length': 128,
 'num_labels': 4}

In [ ]:
json.dump(model_config,open(os.path.join('/content/drive/MyDrive/NLP/ADE/Model-Dec7',"model_config.json"),"w"))